In [1]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from  sklearn.metrics.pairwise import pairwise_distances
import matplotlib
import shutil

SILHOUETTE_DELTA = 0.01 

HIGH_LEVEL_FEATURES = ['script_url','canvas_fingerprinting','canvas_font_fingerprinting', \
                       'audio_context_fingerprinting','webrtc_fingerprinting', \
                       'battery_fingerprinting','triggers_requests', \
                       'triggers_third_party_requests','easylist_blocked', \
                       'easyprivacy_blocked', 'disconnect_blocked', 'third_party_script','min_rank', \
                       'num_sites','num_sites_overall','category','country']

PROB_THRES = 0.7
BATCH_SIZE = 5


In [12]:
## classify misc labelled results 
def classification(data, labels, count, path, thres, limit=None):
    #prepare data
    misc = data['script_url'].as_matrix()
    # drop higher level features 
    df = data.copy()
    for i in HIGH_LEVEL_FEATURES:
        df = df.drop(i, 1)
    df = df.as_matrix()
    clusters = np.copy(labels)
    clf2 = RandomForestClassifier(n_estimators=100, max_features='auto') # sqrt(Num of feature to sample)
    
    print ("Before classifying misc. samples: %d" % len(np.where(clusters == -1)[0]))
    f = np.hstack((np.double(df),np.vstack(clusters)))
    misc = misc[np.where(f[:,-1] == -1)[0]]
    X_train = f[f[:,-1] >= 0, 0:-1]
    y_train = f[f[:,-1] >= 0, -1]
    X_test = f[f[:,-1] == -1, 0:-1]
    y_test = f[f[:,-1] == -1, -1]
    clf2.fit(X_train, y_train)
    res = clf2.predict(X_test)
    prob = clf2.predict_proba(X_test)
    max_prob = np.max(prob, axis=1) # only take the max prob value
    for i in range(min(limit,len(prob))):
        # threshold this can vary for diff feature compression and algorithm
        ind = np.argmax(max_prob)
        if max_prob[ind] > thres: 
            y_test[ind] = res[ind]
            max_prob[ind] = 0.0 # replace the prob
    clusters[clusters == -1] = y_test

    sorted_label = map(lambda x: x, sorted(zip(misc, res, np.max(prob, axis=1)), key=lambda x: x[2], reverse=False))
    df = pd.DataFrame(sorted_label)
    df.to_csv(path+'/script_labels_prob_'+str(count)+'.csv', index=False, header=['url', 'pred', 'prob'])
    print ("After classifying misc. samples: %d" % len(np.where(clusters == -1)[0]))
    return clusters

In [3]:
## plot per cluster silhouette coefficient
def plot_clusterwise_silhouette(data, cluster_labels, widht, height, min_v, max_v):
    matplotlib.rcParams['ps.useafm']=True
    matplotlib.rcParams['pdf.use14corefonts']=True
    matplotlib.rcParams['text.usetex']=True
    #prepare data
    df = data.as_matrix()
        
    # unique clusters
    n_clusters = len(set(cluster_labels))
    
    plt.figure(figsize=(widht, height))
    # get axis for plot
    ax = plt.gca()

    # The silhouette coefficient can range from -1, 1 but in this example all
    ax.set_xlim([min_v, max_v])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax.set_ylim([0, len(df) + (n_clusters + 1) * 10])

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(df, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)
    
    # withour clauster -1 (misc cluster)
    inds = np.where(cluster_labels >= 0)[0]
    silhouette_avg_nomisc = silhouette_score(df[inds,:], cluster_labels[inds])

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(df, cluster_labels)

    y_lower = 10
    for i in sorted(set(cluster_labels)):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.spectral(float(i) / n_clusters)
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        if i >= 0:
            ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        else:
            ax.text(0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    #ax.set_title("The silhouette plot for the various clusters.")
    ax.set_xlabel("Silhouette coefficient values")
    ax.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax.axvline(x=silhouette_avg, color="red", linestyle="--")
    ax.axvline(x=silhouette_avg_nomisc, color="blue", linestyle="-.")
    
    ax.set_yticks([])  # Clear the yaxis labels / ticks
    ax.set_xticks(np.arange(min_v, max_v+0.1, 0.2))
    #plt.show()
    return plt

In [4]:
## check if clusters can be combined without reducing too much of silhouette coefficient
def pairwise_cluster_comparison(data, clusters):
    pairwise_merge = {}
    df = data.as_matrix()
    unique_lables = sorted(set(clusters))
    for i in unique_lables:
        for j in unique_lables:
            if i == j or i == -1 or j == -1:
                continue
            labels = np.copy(clusters) # restore original to make comparison   
            labels[labels == j] = i
            inds = np.where(labels >= 0)[0] # only properly clustered items
            val = silhouette_score(df[inds,:], labels[inds])
            pairwise_merge[(i,j)] = val
    return pairwise_merge

In [5]:
## perform dbscan clustering 
def dbscan_cluster(data, e, s, m, a):
    df = data.as_matrix()
    ## cosine isn't directly supported 
    if  m == 'precomputed':
        df = pairwise_distances(df, metric='cosine')
    dbscan = DBSCAN(eps=e, min_samples=s, metric=m, algorithm=a)
    dbscan.fit(df)
    print  "DBSCAN clusters: ",len(set(dbscan.labels_))
    return dbscan.labels_

In [6]:
## print smaples per cluster
def print_cluster_freq(labels):
    freq = Counter(labels)
    print sorted(freq.items())

In [7]:
## main computation starts here
feats = None
pd.set_option("display.max_colwidth",500)
pd.set_option("display.max_rows",500)
feats = pd.read_csv('feature_file.csv', sep='\t')
print feats.shape  

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (509) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(405937, 514)


In [8]:
interested_scripts=feats[(feats.addEventListener_devicelight == 1) |
      (feats.addEventListener_devicemotion == 1) |
      (feats.addEventListener_deviceorientation == 1) |
      (feats.addEventListener_deviceproximity == 1) ]


# drop higher level features 
interested_scripts_features = interested_scripts.copy()
for i in HIGH_LEVEL_FEATURES:
    interested_scripts_features = interested_scripts_features.drop(i, 1)
    
print interested_scripts_features.shape


(916, 497)


In [9]:
## Using distance metrics for boolean features like dice, jaccard, hamming distance
labels = dbscan_cluster(interested_scripts_features, 0.1, 3, 'dice', 'auto') # may want to tweak the distance threshold
 
font = {'family' : 'Times New Roman',
        'weight' : 'bold',
        'size'   : 22
       }

matplotlib.rcParams['axes.titlesize'] = 40
matplotlib.rcParams['axes.labelsize'] = 30  
matplotlib.rc('font', **font)

if os.path.isdir('Clustering_Result'):
    shutil.rmtree('Clustering_Result')
os.mkdir('Clustering_Result')

plts = plot_clusterwise_silhouette(interested_scripts_features, labels, 15, 15, -0.8, 1)
plts.tight_layout()
plts.savefig('Clustering_Result/cluster_step1.pdf', format='pdf', dpi=1200)
#plts.show()
plts.close()

DBSCAN clusters:  39
('For n_clusters =', 39, 'The average silhouette_score is :', 0.53928478469126995)


In [10]:
## Merge clusters based on silhouette coefficient, considering SILHOUETTE_DELTA as possible noise 

import matplotlib 
font = {'family' : 'Times New Roman',
        'weight' : 'bold',
        'size'   : 22
       }
matplotlib.rcParams['axes.titlesize'] = 40
matplotlib.rcParams['axes.labelsize'] = 30  
matplotlib.rc('font', **font)

plts = None

first_max = None
last_max = None
count = 1
olabels = np.copy(labels)
print ("Original label frequency")
print_cluster_freq(olabels)
while True:
    print ("Round: %d" % count)
    res = pairwise_cluster_comparison(interested_scripts_features, olabels)
    last_max = max(res.values())
    maxs = [i for i, j in res.items() if j == last_max]
    print maxs, last_max
    if first_max == None:
        first_max = last_max
    if first_max - last_max < SILHOUETTE_DELTA: #tolerating 5% as noise
        print  ("max avg_silhouette: %f" % last_max)
        largest_cluster = 0
        selected = None
        for x,y in maxs:
            f1 = len(np.where(olabels == x)[0])
            f2 = len(np.where(olabels == y)[0])
            if largest_cluster < max(f1, f2):
                if f1 > f2:
                    selected = (x, y)
                else:
                    selected = (y, x)
                largest_cluster = max(f1, f2)
        olabels[olabels == selected[1]] = selected[0]
        print ("Merged cluster %d with cluster %d" %(selected[1],selected[0]))
        print_cluster_freq(olabels)
        plts = plot_clusterwise_silhouette(interested_scripts_features, olabels, 15, 15, -0.8, 1)
    else:
        break
    count += 1

plts.tight_layout()
plts.savefig('Clustering_Result/cluster_step2.pdf', format='pdf', dpi=1200)
#plts.show()
plts.close()

Original label frequency
[(-1, 216), (0, 71), (1, 174), (2, 3), (3, 12), (4, 46), (5, 12), (6, 6), (7, 71), (8, 10), (9, 8), (10, 13), (11, 9), (12, 8), (13, 6), (14, 10), (15, 4), (16, 30), (17, 16), (18, 15), (19, 5), (20, 27), (21, 18), (22, 4), (23, 3), (24, 3), (25, 9), (26, 17), (27, 19), (28, 5), (29, 5), (30, 14), (31, 12), (32, 6), (33, 10), (34, 4), (35, 5), (36, 6), (37, 4)]
Round: 1
[(24, 16), (16, 24)] 0.814422455212
max avg_silhouette: 0.814422
Merged cluster 24 with cluster 16
[(-1, 216), (0, 71), (1, 174), (2, 3), (3, 12), (4, 46), (5, 12), (6, 6), (7, 71), (8, 10), (9, 8), (10, 13), (11, 9), (12, 8), (13, 6), (14, 10), (15, 4), (16, 33), (17, 16), (18, 15), (19, 5), (20, 27), (21, 18), (22, 4), (23, 3), (25, 9), (26, 17), (27, 19), (28, 5), (29, 5), (30, 14), (31, 12), (32, 6), (33, 10), (34, 4), (35, 5), (36, 6), (37, 4)]
('For n_clusters =', 38, 'The average silhouette_score is :', 0.5385478969478662)
Round: 2
[(6, 2), (2, 6)] 0.811029919856
max avg_silhouette: 0.811

In [14]:
## see if the misc. samples (labelled as -1) can be classified as one of the other clsuters with high probability
## Random Forest classifiers (with reduced feature sampling) can potentially be more robust to variance 
## here we consider classification prob >= 0.7 (may want to tweak this parameter) and we insert data in batches of 5 

rlabels = np.copy(olabels)
final_label = None
count = 1
while True:
    print ("Round: %d" % count)
    nlabels = classification(interested_scripts, rlabels, count, 'Clustering_Result', 0.8, BATCH_SIZE)
    if np.array_equal(nlabels, rlabels):
        final_label = rlabels
        break
    else:
        rlabels = nlabels
    count += 1
    
## final clustering results
font = {'family' : 'Times New Roman',
        'weight' : 'bold',
        'size'   : 22
       }
matplotlib.rcParams['axes.titlesize'] = 40
matplotlib.rcParams['axes.labelsize'] = 30  
matplotlib.rc('font', **font)

plts = plot_clusterwise_silhouette(interested_scripts_features, final_label, 15, 15, -0.8,1)
plts.tight_layout()
plts.savefig('Clustering_Result/cluster_step3.pdf', format='pdf', dpi=1200)
#plts.show()
plts.close()

Round: 1
Before classifying misc. samples: 216
After classifying misc. samples: 211
Round: 2
Before classifying misc. samples: 211
After classifying misc. samples: 206
Round: 3
Before classifying misc. samples: 206
After classifying misc. samples: 201
Round: 4
Before classifying misc. samples: 201
After classifying misc. samples: 196
Round: 5
Before classifying misc. samples: 196
After classifying misc. samples: 192
Round: 6
Before classifying misc. samples: 192
After classifying misc. samples: 191
Round: 7
Before classifying misc. samples: 191
After classifying misc. samples: 190
Round: 8
Before classifying misc. samples: 190
After classifying misc. samples: 190
('For n_clusters =', 36, 'The average silhouette_score is :', 0.54561983792195601)
